# Sendy Logistics Regression Model

![Image of Yaktocat](https://cdn1.vc4a.com/media/2015/12/Sendy-delivery-900x322.jpg)

## Introduction
Logistics is fundamental to the success of a business while efficient and affordable logistics are a vital component of economic development. Like any logistics company, Sendy aims to improve the efficiency of businesses by providing faster delivery of their products.

### About Sendy:
Sendy is a business-to-business platform established in 2014, to enable businesses of all types and sizes to transport goods more efficiently across East Africa. It was headquartered in Kenya and has gradually expanded across the country and East African borders where  the it is enabling businesses to move large volumes of goods.

### Aim and objectives:
The aim of this notebook is to help Sendy improve their logistics and communicate an accurate arrival time to their customers. This is done through building a prediction model that estimates time of delivery of orders, from the point of driver pickup to the point of arrival at final destination. The model will enhance customer communication and improve the reliability of Sendy's services; which will ultimately improve customer experience. In addition, the solution from the model will enable Sendy to realise cost savings, and ultimately reduce the cost of doing business, through improved resource management and planning for order scheduling.

## Importing the Libraries

In [ ]:
import numpy as np 
import math
# data processing
import pandas as pd 

# data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn import tree

#Varrible selection
from statsmodels.graphics.correlation import plot_corr
from statsmodels.formula.api import ols
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import GridSearchCV

## Getting the Data

In [ ]:
# the csv files are converted to panda dataframe and are renamed.
Train_df= pd.read_csv("Train.csv")
Test_df= pd.read_csv("Test.csv")
Riders_df= pd.read_csv("Riders.csv")
VariableDefinitions_df= pd.read_csv("VariableDefinitions.csv")
SampleSubmission_df= pd.read_csv("SampleSubmission.csv")


## Data Description

Exploring data sets helps in developing deep understanding about the data, the data set is Explored with imported libraries.The Variable dataframe below displays the name of features and desciptions on the right.

In [ ]:
VariableDefinitions_df

The dataframe above is the list of the features with a short description:

In [ ]:
VariableDefinitions_df.info()

In [ ]:
#Train data set
Train_df.info()

The training-set has 21201 entries and 28 features + the target variable (Time from Pickup to Arrival). 6 of the features are floats, 13 are integers and 10 are objects.Since there is a Train data set, this implies that the data has been split and any aditing that will be done on the train data set will also be applied on the Test data set.

In [ ]:
Train_df.head(8)

From the table above,few things can be noted. some of the features need to be converted into numeric ones, so that the machine learning algorithms can process them. Furthermore. Some more features, that contain missing values (NaN = not a number) can also be spoted, that would be deal with later on.

In [ ]:
Train_df.columns.values

Above are all features present in the Train data frame

In [ ]:
    # Rider data set
Riders_df.info()

Rider_df displays the information about the motorbike riders. It consist of 5 features and 960 entries.

In [ ]:
Riders_df.head(8)

Above is the overview of the Riders data frame. The feature (Rider Id) is common to both Riders and Train Data Frame this allows for a merge between the two data frames.

In [ ]:
#features found within the Rider data set.
Riders_df.columns.values

In [ ]:
    #a look at the submission sample data frame
SampleSubmission_df.head(8)

The sample submission displays the format in which sumbmision would be made for this predict on Zindi

## __DATA PREPROCESSING__

### Data cleaning and formating

In [ ]:
    #Joining the riders to both Train and Test data set
Train_df = pd.merge(Train_df, Riders_df, left_on='Rider Id', 
                    right_on='Rider Id', how='left')
Test_df = pd.merge(Test_df, Riders_df, left_on='Rider Id',
                    right_on='Rider Id', how='left')

Test and Train data frames are merged with Riders data data frame on the Rider Id feature because the features in the Rider data frame may have influence on the predict variable.

In [ ]:
Train_df.head(8)

Above is the new merged Train data set and The same is be done for the Test data set. For the purpose of this predict, the Vehicle Type feature is dropped because only motor bikes are considered. The naming format is not consistant in the above data frame(under scores will be use instead of spaces) and Features User Id, Rider ID and Order number will be dropped but the Order No from the Test df will not be dropped since it is require dor the submission.

In [ ]:
print('There are', Train_df['Rider Id'].nunique(), 
      'Motorbike riders', 'and', Train_df['User Id'].nunique(), 
      'Customers')

In [ ]:
    # Formatting the naming of the columns
Train_df.columns = Train_df.columns.str.replace(' ', '_')
Test_df.columns = Test_df .columns.str.replace(' ', '_')

    #removing "-" from the feature labels.
Train_df.columns = Train_df.columns.str.replace('_-_', '_')
Test_df.columns = Test_df .columns.str.replace('_-_', '_')


In [ ]:
Train_df.info()

Accordin to data from Sendy; there is a missing time features 'Arrival at Destination Times' in  the Test df and it can not be culculated because the Time_from_Pickup_to_Arrival.As a result,  Arrival at Destination Times will all be dropped aswel as the Vehicle type. 

In [ ]:
Train_df = Train_df.drop(['Vehicle_Type', 
                          'Arrival_at_Destination_Day_of_Month',
                          'Arrival_at_Destination_Weekday_(Mo_=_1)',
                          'Arrival_at_Destination_Time'], axis = 1)
Test_df = Test_df.drop(['Vehicle_Type'], axis = 1)

In [ ]:
Train_df.info()

__Handling missing data__

In [ ]:
aval_df = Train_df.notnull().sum(axis=0).reset_index()
aval_df.columns = ['column_name', 'missing_count']
aval_df = aval_df.loc[aval_df['missing_count']>0]
aval_df = aval_df.sort_values(by='missing_count')

ind = np.arange(aval_df.shape[0])
width = 0.9
fig, ax = plt.subplots(figsize=(10,8))
rects = ax.barh(ind, aval_df.missing_count.values, color='magenta')
ax.set_yticks(ind)
ax.set_yticklabels(aval_df.column_name.values, rotation='horizontal')
ax.set_xlabel("Total Count of Available values")
ax.set_ylabel("Features")
ax.set_title("Available Data In Each Column")
plt.show()

The chart above display how much data is available, the full bars means the data for that feature is comple and the last two bar bars are the only ones with missing data.

__More detailed look at what data is actually missing:__

In [ ]:
 #Train_df missing data
total = Train_df.isnull().sum().sort_values(ascending=False)
percent_1 = Train_df.isnull().sum()/Train_df.isnull().count()*100
percent_2 = (round(percent_1, 1)).sort_values(ascending=False)
missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%'])
missing_data.head(5)

In [ ]:
#Visual Display of missing Data
ax = missing_data['Total'].plot(kind='barh', figsize=(10, 15), color='#86bf91', zorder=2, width=0.68)
    # Set x-axis label
ax.set_xlabel("Total count of missing values", labelpad=20, weight='bold', size=12)

  # Set y-axis label
ax.set_ylabel("Features", labelpad=20, weight='bold', size=12)

As obseved from both missing_data frame and the visual display above, it will be a bit tricky to deal with the 'Temparature' feature, which has 4366 missing values. However on the other hand, the 'Precipitation in mm' feature needs further investigation, but it looks like it would be better if it is dropped off from the dataset, since 97.4 % of it is missing data.

__Investigating on Precipitation in millimeter feature.__

In [ ]:
    #Checking if there are 0 mm records of Precipitation from both Test and Train data frame
(Train_df['Precipitation_in_millimeters']==0).all(),(Test_df['Precipitation_in_millimeters']==0).all()


The are no Recorded 0 values for precipitation in both of the data frames. It is not likely that pricipitation may occur in all days of the month. So it makes sense that 97% of the time in a month there is no precipitation, therefore the NaN values will be replaced with 0 values.Meaning on those days there was no precipitation.

In [ ]:
    #replacing NAN with 0 for precipitation feature
Train_df["Precipitation_in_millimeters"] = Train_df["Precipitation_in_millimeters"].fillna(0)
Test_df["Precipitation_in_millimeters"] = Test_df["Precipitation_in_millimeters"].fillna(0)

Checking for relatinship between Temparature and features which may have influence of Time from Pickup to Arrival

In [ ]:
    #Train_df data set 
Temp = sns.boxplot(x="Platform_Type", y="Temperature", hue="Personal_or_Business",
                 data=Train_df, palette="Greens")


The whisker and the box diagram propose that there is a relationship between the temparature and the platform as well as the Personal or bussiness feature.

In [ ]:
    #repacing the missing temparature with the mean
Train_df = Train_df.fillna(Train_df.mean())
Test_df = Test_df.fillna(Test_df.mean())

The above will affect only the Temparature since it is the onlyone with the missing values 

__*Data formating*__

There are Four features containg time data but as an object:

- Placement_Time
- Confirmation - Time
- Arrival_at_Pickup_Time
- Pickup_Time

In [ ]:
def alter_time(df):
    time_matrix = ['Placement_Time','Confirmation_Time', 
                   'Arrival_at_Pickup_Time', 'Pickup_Time']
    for i in time_matrix:
        df[i] = pd.to_datetime(df[i]).dt.strftime('%H:%M:%S')
        df[i] = pd.to_timedelta(df[i])
        df[i] = df[i].dt.total_seconds()
        
    return df

In [ ]:
Train_df = alter_time(Train_df)
Test_df = alter_time(Test_df)

In [ ]:
Train_df.head(2)

## Feature engineering

In [ ]:
Train_df['Placement_to_Confiration_Time'] = Train_df['Confirmation_Time'] - Train_df['Placement_Time']
Test_df['Placement_to_Confiration_Time'] = Test_df['Confirmation_Time'] - Test_df['Placement_Time'] 

Creating new time features from the existing ones by culculating the difference from each.
- Placement_Time
- Confirmation - Time
- Arrival_at_Pickup_Time
- Pickup_Time

In [ ]:
Train_df['Placement_to_Confiration_Time'] = Train_df['Confirmation_Time'] - Train_df['Placement_Time']
Test_df['Placement_to_Confiration_Time'] = Test_df['Confirmation_Time'] - Test_df['Placement_Time'] 

Train_df['Placement_to_Arrival_at_Pickup_Time'] = Train_df['Arrival_at_Pickup_Time'] - Train_df['Placement_Time']
Test_df['Placement_to_Arrival_at_Pickup_Time'] = Test_df['Arrival_at_Pickup_Time'] - Test_df['Placement_Time'] 

Train_df['Placement_to_Pickup_Time'] = Train_df['Pickup_Time'] - Train_df['Placement_Time']
Test_df['Placement_to_Pickup_Time'] = Test_df['Pickup_Time'] - Test_df['Placement_Time'] 

Train_df['Confirmation_to_Arrival_at_Pickup_Time'] = Train_df['Arrival_at_Pickup_Time'] - Train_df['Confirmation_Time']
Test_df['Confirmation_to_Arrival_at_Pickup_Time'] = Test_df['Arrival_at_Pickup_Time'] - Test_df['Confirmation_Time'] 

Train_df['Confirmation_to_Pickup_Time'] = Train_df['Confirmation_Time'] - Train_df['Placement_Time']
Test_df['Confirmation_to_Pickup_Time'] = Test_df['Confirmation_Time'] - Test_df['Placement_Time'] 

Train_df['Arrival_at_Pickup_to_Pickup_Time'] = Train_df['Confirmation_Time'] - Train_df['Placement_Time']
Test_df['Arrival_at_Pickup_to_Pickup_Time'] = Test_df['Confirmation_Time'] - Test_df['Placement_Time'] 

In [ ]:
Train_df.head(8)

#### Creating a feature from Coordinates

The haversine formula determines the great-circle distance between two points on a sphere given their longitudes and latitudes.
Below is the fuction that will calculate the disdance between the pickup and destination location in Kilo meters

In [ ]:
def haversine_vectorize(lon1, lat1, lon2, lat2): 
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2]) 
    newlon = lon2 - lon1
    newlat = lat2 - lat1 
    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2
 
    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM(radius of the Earth)
    return round(km, 0)

In [ ]:
distance_1 = haversine_vectorize(Train_df['Pickup_Lat'], 
                               Train_df['Pickup_Long'], 
                               Train_df['Destination_Lat'], 
                               Train_df['Destination_Long'])
Train_df['Actual_Distance_KM'] = distance_1
distance_2 = haversine_vectorize(Train_df['Pickup_Lat'], 
                               Train_df['Pickup_Long'], 
                               Train_df['Destination_Lat'], 
                               Train_df['Destination_Long'])
Test_df['Actual_Distance_KM'] = distance_2

In [ ]:
Train_df.head()

# EDA

Exploring the features in the data frame

#### A first look at the behaviour of delivery time (Y - Variable)

In [ ]:
plt.figure(figsize=(6,6))
plt.scatter(range(Train_df.shape[0]), np.sort(Train_df['Time_from_Pickup_to_Arrival'].values), 
            color = 'orange')
plt.xlabel('X - Variables', fontsize=12)
plt.ylabel('Pickup to Arrival Time', fontsize=12)
plt.show()

 ## *Thasamy will add interpretation*

We are going to look at the Platform Type and Business type to look at what makes up most of the orders.

## Understanding The Distribution Of The Data
We are going to look at the features contained in the dataframe independently.This will help with understanding the distribution of the ranges and/or types contained in each.

### Categorical data

A look at the Platform Type and Business or Personal features to see at what makes up most of the orders.

In [ ]:
Train_df['Platform_Type'].value_counts().plot(kind='pie')
plt.axis('equal')
plt.title('Platform Type contribution to total orders')
plt.ylabel('')
plt.show()

ax=sns.countplot(x='Personal_or_Business', data=Train_df)
ax.set(xlabel='Personal or Business', ylabel='Number of orders')
plt.show()

From the two figures, one can see that platfrom 3 has the highest numbers of orders and platform 4 has the least numbers. It can also be seen from the barchart that there are more orders for business than there are for personal.

## Numerical data

### Placement and confirmation Day of the month

Placement and confirmation day of the month can be the same. If they are, it will save time that only one of the features is looked at. To check that, a new dataframe is created with rows that pass the condition that confirmation day is the same as placement day of the month. After it is created, if its length is still 21201 then this proves they are the same.

In [ ]:
New_df=Train_df[Train_df['Placement_Day_of_Month']!=Train_df['Confirmation_Day_of_Month']]
New_df

In [ ]:
New_df=Train_df[Train_df['Placement_Weekday_(Mo_=_1)']!=Train_df['Confirmation_Weekday_(Mo_=_1)']]
New_df

interptre the above

In [ ]:
sns.distplot(Train_df['Placement_Day_of_Month'],bins=4,kde=False, color='red')
plt.xlabel('Day of the month')
plt.ylabel('Number of orders')
plt.rcParams["patch.force_edgecolor"] = True
plt.show()

interpretation chego

### Distance

The distance is looked at to see the range that contributes the most to the orders. The distance (Km) is divided into 5 ranges and is shown below.

In [ ]:
sns.distplot(Train_df['Distance_(KM)'], bins=np.arange(0,60,10),kde=False, color='green')
ax.set(xlabel='Distance(Km)', ylabel='Number of orders')
plt.rcParams["patch.force_edgecolor"] = True
plt.xlabel('Distance')
plt.ylabel('Number of orders')
plt.show()

The figure above shows that most of the orders are for a traveling distance of between 0km and 10km. Few orders are from a range 30km and 40km.

### Temparature and Precipitation
Now looking at both the temperature data and precipitation data, it can be seen that these are the only features with missing values. The graphs for boths are shown below.

In [ ]:
fig, axes = plt.subplots(1, 2,figsize=(15,5))


sns.distplot(Train_df['Temperature'],ax=axes[0], bins=np.arange(0,60,10),kde=False, color='orange')
ax.set(xlabel='Temperature', ylabel='Number of orders')
plt.rcParams["patch.force_edgecolor"] = True
plt.xlabel('Distance')
plt.ylabel('Number of orders')

sns.distplot(Train_df['Precipitation_in_millimeters'],ax=axes[1], bins=np.arange(0,60,10),kde=False, color='pink')
ax.set(xlabel='Precipitation in millimeters', ylabel='Number of orders')
plt.rcParams["patch.force_edgecolor"] = True
plt.xlabel('Distance')
plt.ylabel('Number of orders')

axes[0].set(xlabel="Temperature", ylabel="Number of orders")
axes[1].set(xlabel="Precipitation in milimeters", ylabel="Number of orders")
fig.subplots_adjust(wspace=0.5)
plt.show()

In [ ]:
#####interprt

In [ ]:
sns.distplot(Train_df['Time_from_Pickup_to_Arrival'],bins=4,kde=False, color='red')
plt.xlabel('Number of orders')
plt.ylabel('Time from Pickup to Arrival')
plt.rcParams["patch.force_edgecolor"] = True
plt.show()

# !!! chego how do you interprete the above.....nb Histogram are normaly used for categorigal data with numeric....

Scatter plot to display the above histplot

In [ ]:
T_PA_no_orders = sns.scatterplot(y="Time_from_Pickup_to_Arrival", x ="No_Of_Orders",  data=Train_df)

the scatter show that the time from pickup to arival does not depend on no of orders

!!!! the above count plot is hard to interprete

### Geography And Coordinates

In [ ]:
pip install ggplot

In [ ]:
from ggplot import *

In [ ]:
color = sns.color_palette()
ggplot(aes(x='Pickup_Lat', y='Pickup_Long', color= 'Time from Pickup to Arrival'), data=Train_df) + \
    geom_point() + \
    scale_color_gradient(low = 'red', high = 'blue')

In [ ]:

ggplot(aes(x='Destination_Lat', y='Destination_Long', color= 'Time_from_Pickup_to_Arrival'), data=Train_df) + \
    geom_point() + \
    scale_color_gradient(low = 'blue', high = 'yellow')

### Observation

The distance of drop-offs are wider spread than pick up points, many destinations arrival time will be influenced by wider distances. We also notice that lengthy times for deliveries occur in both the central and outlier destinations. Traffic congestion and distance are all influential in the pick to arrival time duration

## Variables and Variable Selection

### Summary Statistics

In this section, the data types and the summary statistics of the variables are explored.
Looking at the data types and number of entries of each column.

In [ ]:
Train_df.info()

The Train_df.info() specifically outputs the number of non-null entries in each column. As such,It can be certain that the data has missing values if columns have a varying number of non-null entries.

Below is the table showcasing the summary statistics of the data.

In [ ]:
Train_df.describe()

Based on the means and standard deviations of different columns, The data will be standardized.

### Dummy Variable Encoding

The User ID, Order_No and Rider Id features will be dropped because they do not influence the Time_from_Pickup_to_Arrival, but the Order No feature will not be dropped from Test_df as it is required for subbmision.

As it can be observed in the above table, the summary statistics being displayed is for numerical data at the moment. More importantly, all input data for regression model building purposes needs to be numerical. Therefore the text data Busines_or_Personal will be transformed  into numbers before training the machine learning model.

To facilitate this transformation from textual-categorical data to numerical equivalents, pandas method called get_dummie will be used. The text data is categorical variable, and get_dummies will transform all the categorical text data into numbers by adding a column for each distinct category. The new column has a 1 for observations which were in this category, and a 0 for observations that were not.

In [ ]:
#droping User_Id, Order_No and Rider Id  Features
Train_df = Train_df.drop(['Order_No', 'User_Id', 'Rider_Id'], axis = 1)
Test_df = Test_df.drop(['User_Id', 'Rider_Id'], axis = 1)

In [ ]:
Train_df.info()

In [ ]:
df_dummies = pd.get_dummies(Train_df)

In [ ]:
df_dummies = pd.get_dummies(Train_df, drop_first=True)

# Making sure that all the column names have correct format
df_dummies.columns = [col.replace(" ", "_") for col in df_dummies.columns]
df_dummies.columns = [col.replace("(Mo_=_1)","Mo_1") for col in df_dummies.columns]
df_dummies.columns = [col.replace("(KM)","KM") for col in df_dummies.columns]
# Test_df
Test_df.columns = [col.replace(" ", "_") for col in Test_df.columns]
Test_df.columns = [col.replace("(Mo_=_1)","Mo_1") for col in Test_df.columns]
Test_df.columns = [col.replace("(KM)","KM") for col in Test_df.columns]

# Reorder columns with the dependent variable (claim_amount) the last column
column_titles = [col for col in df_dummies.columns if col !=
                 'Time_from_Pickup_to_Arrival'] + ['Time_from_Pickup_to_Arrival']
df_dummies = df_dummies.reindex(columns=column_titles)

df_dummies.head()

In [ ]:
df_dummies.shape

### Correlations and Model Structure

Using the dummy variable dataframe, a model that predictsTime from Pickup to Arrival (the dependent variable) as a function of 33 different independent variables (IVs) can be build.

Before this can be done, however, its better to reorder columns so that the dependent variable is the last column of the dataframe. This will make a heatmap visualisation representing a correlation matrix of the data easier to interpret.

In [ ]:
column_titles = [col for col in df_dummies.columns if col!= 'Time_from_Pickup_to_Arrival'] + ['Time_from_Pickup_to_Arrival']
df_dummies=df_dummies.reindex(columns=column_titles)

In [ ]:
df_dummies.corr()

In [ ]:
fig = plt.figure(figsize=(15,15));
ax = fig.add_subplot(111);
plot_corr(df_dummies.corr(), xnames = df_dummies.corr().columns, ax = ax);

If all of these variables are to be used, it would be like effectively working with more than enough information. The model will also have collinearity issues:

Peronal_or_Business_Personal and peronal_or_Business_Business are perfectly negative correlated. All the features having blue squeres are also perfectly negative correlated.
This will likely be a problem when building a model.

checking what an OLS model summary says.

## Fitting the model using statsmodels.OLS

#### Generating the regression string
Following the process initially detailed in the Multiple Linear Regression Pt 2 - Checking Model Quality train, OLS model will be build and the model summary will be printed:

y ~ X

which is read as follows: "Regress y on X". statsmodels works in a similar way, so an appropriate string need to be generated to feed to the method in case it is required to fit the model.

In [ ]:
# Model DataFrame with all of the columns:
dfm = df_dummies.copy()

# The dependent variable:
y_name = 'Time_from_Pickup_to_Arrival'
# The independent variable
# (let's first try all of the columns in the model DataFrame)
X_names = [col for col in dfm.columns if col != y_name]

# Build the OLS formula string " y ~ X "
formula_str = y_name+" ~ "+" + ".join(X_names);
print('Formula:\n\t {}'.format(formula_str))

In [ ]:
# Fitting the model using the model dataframe
model=ols(formula=formula_str, data=dfm)
fitted = model.fit()

# Output the fitted summary
print(fitted.summary())

There is warning about strong multicollinearity. This is likely as a result of the incorrect filtering of one hot encoded dummy variables (It was noticed earlier that Peronal_or_Business_Personal and peronal_or_Business_Business are perfectly negative correlated).

In order to ensure an underlying relationship between the categories is not assumed, pd.get_dummies is called with the argument drop_first=True so that only  n-1 columns are create for each variable with n categories.

In [ ]:
df_dummies = pd.get_dummies(Train_df, drop_first=True)

# Again make sure that all the column names have underscores instead of whitespaces
df_dummies.columns = [col.replace(" ", "_") for col in df_dummies.columns]
df_dummies.columns = [col.replace("(Mo_=_1)","Mo_1") for col in df_dummies.columns]
df_dummies.columns = [col.replace("(KM)","KM") for col in df_dummies.columns]

# Reorder columns with the dependent variable (claim_amount) the last column
column_titles = [col for col in df_dummies.columns if col !=
                 'Time_from_Pickup_to_Arrival'] + ['Time_from_Pickup_to_Arrival']
df_dummies = df_dummies.reindex(columns=column_titles)

df_dummies.head()

In [ ]:
df_dummies.shape

Now there are 28 columns instead of 39. This gives for 28 potential independent variables that could be used to build a relationship on Time_from_Pickup_to_Arrival

Checking what the OLS model summary would say if now only the 30 variable columns are fit

In [ ]:
# The model DataFrame will be kept, but only specify the columns wanted to fit this time
X_names = [col for col in df_dummies.columns if col != y_name]

# Build the OLS formula string " y ~ X "
formula_str = y_name+' ~ '+'+'.join(X_names)

# Fit the model using the model dataframe
model = ols(formula=formula_str, data=dfm)
fitted = model.fit()

# Output the fitted summary
print(fitted.summary())

The condition number has improved, but there is still mention of strong multicollinearity in warning [2]

Making further selections on the variables now using their significance.



## Variable Selection by Correlation and Significance

now there are 30 predictor variables to choose from, a way of guiding is needed to choose the best ones to be predictors. One way is to look at the correlations between the Time_from_Pickup_to_Arrival and each variables in the DataFrame and select those with the strongest correlations (both positive and negative).

To consider how significant those features are.
The code below will create a new DataFrame and store the correlation coefficents and p-values in that DataFrame for reference.

In [ ]:
# Calculating correlations between predictor variables and the response variable
corrs = df_dummies.corr()['Time_from_Pickup_to_Arrival'].sort_values(ascending=False)

In [ ]:

# a dictionary of correlation coefficients and p-values
dict_cp = {}

column_titles = [col for col in corrs.index if col!= 'Time_from_Pickup_to_Arrival']
for col in column_titles:
    p_val = round(pearsonr(df_dummies[col], df_dummies['Time_from_Pickup_to_Arrival'])[1],6)
    dict_cp[col] = {'Correlation_Coefficient':corrs[col],
                    'P_Value':p_val}

df_cp = pd.DataFrame(dict_cp).T
df_cp_sorted = df_cp.sort_values('P_Value')
df_cp_sorted[df_cp_sorted['P_Value']<0.1]

Above is the sorted list of the p-values and correlation coefficients for each of the features, when considered on their own.

If a logic test was to be used with a significance value of 5% (p-value < 0.05),it would imply that the following features are statistically significant:
- Distance_KM
- Average_Rating
- the Coordinates (pick long and lat, dest long and lat)
- No-of_Orders
- Arrival_at_Destination_time
- Actual_Distance
- Placement_to_Destination_Time
- Arrived_at_Pickup_Time
- Placement_Day_od_Month
- Confirmation_Day_of_Month
- Pickup_Day_Of_Month
- Arrival_at_Pickup_Day_of_Month


the variables that have a significant correlation with the dependent variable are kept.they will be put into an independent variable DataFrame X

In [ ]:
# The dependent variable remains the same:
y_data = df_dummies[y_name]  # y_name = 'Time_from_Pickup_to_Arrival'

# Model building - Independent Variable (IV) DataFrame
X_names = list(df_cp[df_cp['P_Value'] < 0.05].index)
X_data = df_dummies[X_names]

To avoid autocorrelation,it is needed to look for predictor variable pairs which have a high correlation with each other.

In [ ]:
# Creating the correlation matrix
corr = X_data.corr()

# Find rows and columnd where correlation coefficients > 0.9 or <-0.9
corr[np.abs(corr) > 0.9]

Instead of looking at the whole correlation matrix, it might be easier to isolate the sections of the correlation matrix to where the off-diagonal correlations are high:

In [ ]:
# just like before, the correlation matrix is created
# rows and columnd where correlation coefficients > 0.9 or <-0.9
corr = X_data.corr()
r, c = np.where(np.abs(corr) > 0.9)

# The interest is in the off diagonal entries:
off_diagonal = np.where(r != c)

# Showing the correlation matrix rows and columns where there is  highly correlated off diagonal entries:
corr.iloc[r[off_diagonal], c[off_diagonal]]

Okay, so it looks like six of the features are very highly correlated.

This is also visible looking back at the correlation coefficient heatmap and matrix from earlier, but a more focused / subset view of the matrix is useful to isolate the coefficients of interest.

Distance is slightly better correlated (and lower p-value) to the dependent variable
there fore; Arrival_at_Pickip_day_of_Month, Pickup_Day_of_Month, Confirmation_Day_of_Month and Placement_Day_of_Month are dropped form the feature dataframe. 
The resulting OLS fit summary:

In [ ]:
# a new subset of potential independent variables
X_remove = ['Arrival_at_Pickip_day_of_Month', 'Pickup_Day_of_Month', 'Confirmation_Day_of_Month', 'Placement_Day_of_Month']
X_corr_names = [col for col in X_names if col not in X_remove]

# Creating new OLS formula based-upon the smaller subset
formula_str = y_name+' ~ '+' + '.join(X_corr_names);
print('Formula:\n\t{}'.format(formula_str))

In [ ]:
# Fit the OLS model using the model dataframe
model=ols(formula=formula_str, data=dfm)
fitted = model.fit()

# Display the fitted summary
print(fitted.summary())

### Variable Selection by Variance Thresholds

Variance Thresholds remove features whose values don't change much from observation to observation. The objective here is to remove all features that have a variance lower than the selected threshold.

It is important to note that variance is dependent on scale, so the features will have to be normalized before implementing variance thresholding.

In [ ]:
# Separating data into independent (X) and independent (y) variables
X_names = list(df_dummies.columns)
X_names.remove(y_name)
X_data = df_dummies[X_names]
y_data = df_dummies[y_name]

In [ ]:
# Normalizing data
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_data)
X_normalize = pd.DataFrame(X_scaled, columns=X_data.columns)

### Variance Threshold in Scikit Learn

To implement Variance Threshold in Scikit Learn:

Import and create an instance of the VarianceThreshold class;
Using the .fit() method to select subset of features based on the threshold.

In [ ]:
# Create VarianceThreshold object
selector = VarianceThreshold(threshold=0.03)

# Using the object to apply the threshold on data
selector.fit(X_normalize)

The Variance Threshold has been applied to the data.Now a closer look at the calculated variance for each predictive variable.

In [ ]:
# Getting column variances
column_variances = selector.variances_

vars_dict = {}
vars_dict = [{"Variable_Name": c_name, "Variance": c_var}
             for c_name, c_var in zip(X_normalize.columns, column_variances)]
df_vars = pd.DataFrame(vars_dict)
df_vars.sort_values(by='Variance', ascending=False)

The above table shows the variances of the individual columns before any threshold is applied. It allows anyone to revise  initial variance threshold if there might be a need to exclude important variables.

The results needs to be extracted and used to select new columns - which form a subset of all the columns.

In [ ]:
# Selecting new columns
X_new = X_normalize[X_normalize.columns[selector.get_support(indices=True)]]

# Saving variable names for later
X_var_names = X_new.columns

# Viewing first few entries
X_new.head()

In [ ]:
X_new.shape

With a threshold of 0.03, the predictor number has gone from 29 to 12 predictors.

Trying few more few more thresholds

In [ ]:
# Creating Variance Threshold objects
selector_1 = VarianceThreshold(threshold=0.05)
selector_2 = VarianceThreshold(threshold=0.1)
selector_3 = VarianceThreshold(threshold=0.15)

In [ ]:
selector_1.fit(X_normalize)

In [ ]:
selector_2.fit(X_normalize)

In [ ]:
selector_3.fit(X_normalize)

In [ ]:
X_1 = X_normalize[X_normalize.columns[selector_1.get_support(indices=True)]]
X_2 = X_normalize[X_normalize.columns[selector_2.get_support(indices=True)]]
X_3 = X_normalize[X_normalize.columns[selector_3.get_support(indices=True)]]

 graphing the number of predictors by the thresholds to investigate the relationship

In [ ]:
f, ax = plt.subplots(figsize=(8, 3), nrows=1, ncols=1)

# Create list of titles and predictions to use in for loop
subset_preds = [X_1.shape[1], X_2.shape[1], X_3.shape[1]]
thresholds = ['0.05', '0.1', '0.15']

# Plot graph
ax.set_title('# of Predictors vs Thresholds')
ax.set_ylabel('# of Predictors')
ax.set_xlabel('Threshold')
sns.barplot(x=thresholds, y=subset_preds)
plt.show()

an increase the threshold, the number of dimensions decrease

checking what the resulting OLS fit summary for a threshold of 0.03 says:

In [ ]:
# new OLS formula
formula_str = y_name+' ~ '+' + '.join(X_new.columns)
print('Formula:\n\t{}'.format(formula_str))

In [ ]:
# Fitting the model using the model dataframe
model = ols(formula=formula_str, data=df_dummies)
fitted = model.fit()

print(fitted.summary())

### modelling the y variable¶
Now that the DataFrame has been shortened using various methods, Its time to see if  linear regression models can be fit and compare them

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

### Preprocess data
Assuring that all models are trained and tested on the same data.

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_data,
                                                    y_data,
                                                    test_size=0.20,
                                                    shuffle=False)

In [ ]:
# Get training and testing data for variance threshold model
X_var_train = X_train[X_var_names]
X_var_test = X_test[X_var_names]

In [ ]:
# Get training and testing data for correlation threshold model
X_corr_train = X_train[X_corr_names]
X_corr_test = X_test[X_corr_names]

Fitting models
instantiate and fit the models

In [ ]:
lm = LinearRegression()
lm_corr = LinearRegression()
lm_var = LinearRegression()

In [ ]:
lm.fit(X_train, y_train);
lm_corr.fit(X_corr_train,y_train);
lm_var.fit(X_var_train,y_train);

#### Assessing model accuracy
checking how the linear models performed!

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Creating figure and axes
f, ax = plt.subplots(figsize=(15, 5), nrows=1, ncols=3, sharey=True)

# Creating list of titles and predictions to use in for loop
train_pred = [lm.predict(X_train),
              lm_corr.predict(X_corr_train),
              lm_var.predict(X_var_train)]
test_pred = [lm.predict(X_test),
             lm_corr.predict(X_corr_test),
             lm_var.predict(X_var_test)]
title = ['No threshold', 'Corr threshold', 'Var threshold']

# Key:
# No threshold - linear regression with all predictive variables
# Corr threshold - linear regression with correlation thresholded predictive variables
# Var threshold - linear regression with variance thresholded predictive variables


# Loop through all axes to plot each model's results
for i in range(3):
    test_mse = round(mean_squared_error(test_pred[i], y_test), 4)
    test_r2 = round(r2_score(test_pred[i], y_test), 4)
    train_mse = round(mean_squared_error(train_pred[i], y_train), 4)
    train_r2 = round(r2_score(train_pred[i], y_train), 4)
    title_str = f"Linear Regression({title[i]}) \n train MSE = {train_mse} \n " + \
                f"test MSE = {test_mse} \n training $R^{2}$ = {train_r2} \n " + \
                f"test $R^{2}$ = {test_r2}"
    ax[i].set_title(title_str)
    ax[i].set_xlabel('Actual')
    ax[i].set_ylabel('Predicted')
    ax[i].plot(y_test, y_test, 'r')
    ax[i].scatter(y_test, test_pred[i])

#  Lasso

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
## Dummy variables
def encode_dummy(df, column_name):
    df[column_name] =  pd.get_dummies(df[column_name],drop_first = True)
    return df

In [ ]:
# import LASSO module
from sklearn.linear_model import Lasso

In [ ]:
# create standardization object
scaler = StandardScaler()

In [ ]:
# create LASSO model object, setting alpha to 0.01
lasso = Lasso(alpha=0.01)

In [ ]:
# split dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled,
                                                    y_data,
                                                    test_size=0.20,
                                                    random_state=1,
                                                    shuffle=False)

In [ ]:
# create LASSO model object, setting alpha to 0.01
lasso = Lasso(alpha=0.01)

In [ ]:
# train the LASSO model
lasso.fit(X_train, y_train)

In [ ]:
# extract intercept from model
intercept = float(lasso.intercept_)

In [ ]:
# extract coefficient from model
coeff = pd.DataFrame(lasso.coef_, Train_df.columns, columns=['Coefficient'])

In [ ]:
intercept = float(lasso.intercept_)

In [ ]:
# extract intercept
print("Intercept:", float(intercept))

In [ ]:
coeff

In [ ]:
# save standardized features into new variable
X_scaled = scaler.fit_transform(X)

In [ ]:
# import train/test split module
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
# create LASSO model object, setting alpha to 0.01
lasso = Lasso(alpha=0.01)

In [ ]:
lasso.fit(X_train, y_train)

In [ ]:
intercept = float(lasso.intercept_)

In [ ]:
coeff = pd.DataFrame(lasso.coef_, X.columns, columns=['Coefficient'])

In [ ]:
print("Intercept:", float(intercept))

In [ ]:
coeff

# Models

In [153]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
def s_v_m(X, y):
    regressor = SVR(kernel = 'rbf')
    regressor.fit(X, y)
    return regressor

In [ ]:
def random_forest(X, y):
    regressor = RandomForestRegressor(n_estimators = 5, random_state = 0)
    regressor.fit(X, y)
    return regressor

In [ ]:
def decision_tree(X, y):
    regressor = DecisionTreeRegressor(random_state = 0)
    regressor.fit(X, y)
    return regressor

In [ ]:
def poly_regression(X, y):
    poly_reg = PolynomialFeatures(degree = 2)
    X_poly = poly_reg.fit_transform(X)
    lin_reg_2 = LinearRegression()
    lin_reg_2.fit(X_poly, y)
    return lin_reg_2

In [ ]:
def multi_linear(X, y):
    regressor = LinearRegression()
    regressor.fit(X, y)
    return regressor

In [154]:
def RMSE_perfomance(y_true, y_pred):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse

In [ ]:
y_train = y_train.values

In [ ]:
y_test = y_test.values

In [149]:
model = multi_linear(X_train, y_train)

In [150]:
y_pred = model.predict(X_test)

In [151]:
y_pred

array([1410.7421875, 2959.8671875, 1662.7421875, ..., 2621.1171875,
       1908.9921875, 1775.7421875])

In [152]:
for i in range(len(y_pred)):
    print(str(y_test[i])+ "\t" + str(y_pred[i]) )

1143	1410.7421875
3115	2959.8671875
2124	1662.7421875
1840	1679.4921875
764	953.2421875
1365	1971.3671875
2143	1951.2421875
1353	674.9921875
395	1064.8671875
556	923.4921875
1622	1414.2421875
1318	1887.4921875
1135	1250.2421875
1210	1877.9921875
2063	2074.7421875
3041	2053.7421875
335	877.2421875
1469	1172.9921875
1875	1648.7421875
1383	1841.7421875
792	1288.9921875
1281	1255.2421875
889	1383.2421875
683	1087.9921875
2236	2903.2421875
2729	1284.4765625
1574	1760.9921875
519	810.2421875
762	1049.1171875
2422	1125.9921875
1393	1831.7421875
704	1210.6171875
2430	2173.2421875
2	963.2421875
3423	2103.8671875
1486	2046.7421875
1858	2613.7421875
1072	2048.4921875
9	1033.9921875
813	1608.9921875
1313	1111.8984375
1074	1804.9921875
1681	1874.4921875
3658	3008.4921875
243	886.4921875
1430	1114.7421875
724	1121.3671875
1116	1336.9921875
1049	1088.1171875
1473	1243.4921875
2542	2589.7421875
1901	1461.7421875
614	1022.7421875
1243	1809.1171875
3551	1846.7421875
1323	1910.8671875
1821	1413.7421875
2

713	1503.2421875
1027	920.2421875
1779	1438.2421875
1223	987.7421875
1175	1863.9921875
1742	2002.4921875
496	773.9921875
1695	2019.7421875
626	1072.1171875
1122	1496.9921875
646	1199.2421875
2340	1108.7421875
281	651.9921875
2807	3011.9921875
1053	2028.6171875
2165	1508.6171875
1358	1159.7421875
1334	1263.9921875
1057	1472.6171875
1370	703.4921875
36	971.4921875
713	1008.7421875
1811	1634.7421875
1208	1374.7421875
718	1185.4921875
521	929.7421875
807	1445.9921875
2345	1141.4921875
2062	2298.2421875
1889	1609.7421875
895	1015.8671875
2540	2099.2421875
2083	827.9921875
650	1061.7421875
634	1025.9921875
1798	1872.9921875
298	763.4921875
5	743.7421875
7	1334.2421875
28	786.7421875
3326	2523.4921875
2111	1703.7421875
1319	1523.7421875
469	885.2421875
2281	1010.1796875
1230	1601.7421875
498	1425.2421875
674	935.7421875
1124	1126.8671875
1551	1082.7421875
1568	1665.2421875
2	426.9921875
1182	1385.4921875
3041	2155.2421875
3	957.9921875
1872	1938.7421875
1975	1270.6171875
917	1650.6171875
2709

956	1185.4921875
1201	1410.8671875
777	883.7421875
2076	2355.3671875
1873	1927.3671875
1097	1228.7421875
494	834.7421875
2505	3125.9921875
2089	1500.2421875
1712	1579.9921875
4219	2983.4921875
404	753.4921875
760	1282.4921875
2062	1508.3671875
1180	1681.7421875
1210	1108.4921875
1601	2445.2421875
2353	1888.1796875
1241	1328.2421875
2662	1357.9921875
1490	947.2421875
4318	2259.4921875
373	726.7421875
3605	1472.6171875
883	711.9921875
1825	1900.2421875
1040	1207.9921875
1429	1544.9921875
1147	1440.7421875
691	1115.7421875
1952	968.2421875
1080	1181.7421875
86	1184.4921875
1115	1133.2421875
693	1326.2421875
2445	1402.4921875
1006	1062.4921875
1331	1498.7421875
1833	1744.4921875
1311	1251.7421875
2356	1281.7421875
1553	2064.6171875
1364	1462.9921875
257	1729.9921875
1753	1520.4921875
2736	2134.2421875
1577	2059.2421875
2805	1455.7421875
544	1087.7421875
692	1427.2421875
248	925.2421875
1479	1488.9921875
2342	1441.7421875
3532	2459.4921875
497	893.9921875
2776	1527.8671875
2160	2312.4921875

3119	1457.2421875
1248	1293.4921875
3698	3151.9921875
105	1769.2421875
819	1152.4921875
1578	1214.2421875
5196	2410.2421875
2466	3068.7421875
15	1041.8671875
1039	1110.3671875
662	861.2421875
1918	2872.7421875
528	1398.2421875
1217	1887.2421875
2345	1578.7421875
4326	2460.9921875
1884	1990.7421875
1173	1296.9921875
1530	1215.7421875
625	1289.7421875
1792	1571.2421875
2391	2304.4921875
1711	1895.4921875
1978	2425.7421875
951	1383.3984375
1254	1587.7421875
1944	2631.2421875
2456	1629.2421875
1616	1364.2421875
1089	1715.7421875
3813	2542.4921875
1138	2095.9921875
1282	1013.7421875
1326	1716.2421875
3625	1392.2421875
2669	1295.2421875
1169	1584.9921875
2032	1555.4921875
1266	1453.7421875
2169	949.8046875
1274	1719.2421875
1106	1698.7421875
1448	1287.4921875
2320	2016.7421875
2	416.2421875
2724	1484.9921875
1424	1088.4921875
1637	1570.9921875
707	1701.9921875
753	847.7421875
1209	1964.7421875
1515	1905.4921875
2217	2115.2421875
2903	1363.7421875
1448	1341.2421875
1059	1500.7421875
247	1130.

In [155]:
print(RMSE_perfomance(y_test, y_pred))

753.5630641087931
